In [1]:
import math
import json
import numpy as np
from collections import Counter

import torch
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import plotly.graph_objects as go

from tqdm import tqdm

from nano_bert.model_ import BertMix3
from nano_bert.tokenizer import WordTokenizer
torch.manual_seed(114514)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
data = None
with open('data/imdb_train.json') as f:
    data = [json.loads(l) for l in f.readlines()]

In [3]:
test_data = None
with open('data/imdb_test.json') as f:
    test_data = [json.loads(l) for l in f.readlines()]

In [4]:
rawvocab = [] # whole vocab
for d in tqdm(data): 
    rawvocab.append([w.lower() for w in d['text']]) # symbol like '.' is remained
vocab = set() # vocab for words appear more than 2 times(minappear = 2)
minappear = 2
for v in tqdm(rawvocab):
    if rawvocab.count(v) > minappear - 1:
        vocab |= set(v)

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:19<00:00, 1253.97it/s]


In [5]:
def encode_label(label):
    if label == 'pos':
        return 1
    elif label == 'neg':
        return 0
    raise Exception(f'Unknown Label: {label}!')


class IMDBDataloader:
    def __init__(self, data, test_data, tokenizer, label_encoder, batch_size, val_frac=0.2):
        train_data, val_data = train_test_split(data, shuffle=True, random_state=1, test_size=val_frac)

        self.splits = {
            'train': [d['text'] for d in train_data],
            'test': [d['text'] for d in test_data],
            'val': [d['text'] for d in val_data]
        }

        self.labels = {
            'train': [d['label'] for d in train_data],
            'test': [d['label'] for d in test_data],
            'val': [d['label'] for d in val_data]
        }

        self.tokenized = {
            'train': [tokenizer(record).unsqueeze(0) for record in
                      tqdm(self.splits['train'], desc='Train Tokenization',position=0)], # divide different sentences in comments
            'test': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['test'], desc='Test Tokenization',position=0)],
            'val': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['val'], desc='Val Tokenization',position=0)],
        }

        self.encoded_labels = {
            'train': [label_encoder(label) for label in tqdm(self.labels['train'], desc='Train Label Encoding',position=0)],
            'test': [label_encoder(label) for label in tqdm(self.labels['test'], desc='Test Label Encoding',position=0)],
            'val': [label_encoder(label) for label in tqdm(self.labels['val'], desc='Val Label Encoding',position=0)],
        }

        self.curr_batch = 0
        self.batch_size = batch_size
        self.iterate_split = None

    def peek(self, split):
        return {
            'input_ids': self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
            'label_ids': self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
        }

    def take(self, split):
        batch = self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': batch,
            'label_ids': labels,
        }

    def peek_tokenized(self, split):
        return {
            'input_ids': torch.cat(
                self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dim=0),
            'label_ids': torch.tensor(
                self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dtype=torch.long),
        }

    def peek_index_tokenized(self, index, split):
        return {
            'input_ids': torch.cat(
                [self.tokenized[split][index]],
                dim=0),
            'label_ids': torch.tensor(
                [self.encoded_labels[split][index]],
                dtype=torch.long),
        }

    def peek_index(self, index, split):
        return {
            'input_ids': [self.splits[split][index]],
            'label_ids': [self.labels[split][index]],
        }

    def take_tokenized(self, split):
        batch = self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': torch.cat(batch, dim=0),
            'label_ids': torch.tensor(labels, dtype=torch.long),
        }

    def get_split(self, split):
        self.iterate_split = split
        return self

    def steps(self, split):
        return len(self.tokenized[split]) // self.batch_size

    def __iter__(self):
        self.reset()
        return self

    def __next__(self):
        if self.batch_size * self.curr_batch < len(self.splits[self.iterate_split]):
            return self.take_tokenized(self.iterate_split)
        else:
            raise StopIteration

    def reset(self):
        self.curr_batch = 0

In [6]:
NUM_CLASS = 2
BATCH_SIZE = 32
MAX_SEQ_LEN = 128
LEARNING_RATE = 1e-4

In [7]:
vocab.discard('.') # '.' is included in taken's vocab enumerate
tokenizer = WordTokenizer(vocab=vocab, max_seq_len=MAX_SEQ_LEN)
tokenizer

Tokenizer[vocab=3665,self.special_tokens=['[MSK]', '[PAD]', '[CLS]', '[SEP]', '[UNK]', '[SOS]', '.'],self.sep=' ',self.max_seq_len=128]

In [8]:
dataloader = IMDBDataloader(data, test_data, tokenizer, encode_label, batch_size=BATCH_SIZE)

Val Label Encoding: 100%|██████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 701107.25it/s]


In [32]:
a = position = torch.arange(0, 128).unsqueeze(0)
a

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127]])

In [37]:
class AbsolutePositionEmbedding(torch.nn.Module): # for DEBERTA
    def __init__(self, n_embed, max_seq_len):
        super(AbsolutePositionEmbedding, self).__init__()
        
        self.max_seq_len = max_seq_len
        
        self.position_embeddings = nn.Embedding(max_seq_len, n_embed)
        
    def forward(self, input_ids):
        # Get the position IDs from the input IDs
        position_ids = torch.arange(0, self.max_seq_len, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        
        # Get the position embeddings
        position_embeddings = self.position_embeddings(position_ids)
        
        return position_embeddings
    

In [63]:
pad_nsp(torch.arange(0, 114).to(device), 128)

tensor([  0.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
         35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,
         47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,
         59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,
         71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,
         83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,
         95.,  96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106.,
        107., 108., 109., 110., 111., 112., 113.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.], device='cuda:0')

In [68]:
a = torch.arange(0, 128).unsqueeze(0)
torch.manual_seed(114514)
Bert = AbsolutePositionEmbedding(2, 128)
b = Bert(a)
Bert = BertEmbeddings(400, 3)
c = Bert(a)
c
Bert = DEBertAEmbeddings(400, 128)
c = Bert(a)
c

tensor([[[ 0.7440, -1.6517, -0.7186, -1.8897],
         [-0.3794,  2.4063,  1.8389, -1.0588],
         [-0.1331,  0.6404, -1.2773,  0.9912],
         [-1.3810,  0.1835, -0.8249, -0.3183],
         [-1.0342, -1.0985, -0.7125,  0.9913],
         [ 0.4465,  0.3917,  0.6880,  0.5020],
         [-1.2138,  0.6493,  1.3062,  1.5970],
         [ 1.0245,  0.1289, -0.0637, -0.8690],
         [-0.8740, -1.3837, -0.8737, -0.9552],
         [-0.4895,  0.0757,  0.1145,  0.5715],
         [ 0.7882,  0.0627,  0.4653, -1.2508],
         [ 1.9917, -0.0231, -2.0122,  0.8020],
         [-0.0132, -0.5354, -0.7686,  0.4104],
         [ 0.2389,  0.1792,  2.2009,  0.2688],
         [-0.7481, -0.9645, -0.1956,  0.4764],
         [-0.3492, -0.8736,  2.0249,  1.1491],
         [ 2.1027,  1.3551, -0.9135,  0.4117],
         [-0.8189,  0.8682,  0.4944, -1.3722],
         [ 0.6548, -2.3145, -2.9876,  1.1823],
         [ 0.0463, -0.7179, -0.6611, -1.3129],
         [-1.7585, -0.3310,  0.7215, -0.0833],
         [ 1.

In [54]:
b

tensor([[[ 1.6500e-01, -4.2874e-01],
         [-2.4267e-02,  1.4942e-01],
         [-7.0685e-02,  2.1661e-01],
         [-1.0899e+00,  8.9571e-01],
         [-3.2904e-01,  1.1737e+00],
         [ 4.5906e-01, -2.0881e+00],
         [-6.8765e-01, -3.2952e-01],
         [-1.7524e-01,  8.3306e-01],
         [-1.6071e+00, -4.6016e-01],
         [ 1.3175e-01,  1.3799e+00],
         [ 1.0430e+00,  1.2995e+00],
         [ 1.9181e+00,  4.7578e-01],
         [ 8.9001e-01, -1.1867e+00],
         [-3.6335e-01, -1.0446e+00],
         [ 8.6211e-01, -5.1553e-01],
         [-7.7988e-01, -1.8068e+00],
         [ 1.4480e+00,  1.1284e+00],
         [-5.3514e-01,  1.3089e+00],
         [ 5.4544e-01,  1.4012e+00],
         [ 1.4425e+00,  8.3782e-01],
         [-1.1620e-01,  9.1472e-01],
         [-2.4817e-01, -5.4873e-01],
         [ 9.2141e-01, -1.0283e+00],
         [-9.4015e-01, -6.1165e-01],
         [ 5.2813e-01,  7.4984e-01],
         [ 2.1591e-01,  8.1243e-01],
         [-1.5864e+00,  9.2640e-01],
 

In [60]:
d = torch.cat((b, c), dim = 2)
d

tensor([[[ 1.6500e-01, -4.2874e-01,  8.4829e-01,  1.0958e+00,  3.9295e-01],
         [-2.4267e-02,  1.4942e-01,  1.6070e+00, -1.4871e+00,  4.5711e-01],
         [-7.0685e-02,  2.1661e-01,  4.2503e-01,  7.3498e-01, -1.0292e+00],
         [-1.0899e+00,  8.9571e-01,  7.1001e-01,  7.9472e-01, -8.1961e-01],
         [-3.2904e-01,  1.1737e+00, -5.3887e-01,  1.3320e+00, -1.0281e+00],
         [ 4.5906e-01, -2.0881e+00, -1.4809e+00, -4.3133e-01,  3.5989e-01],
         [-6.8765e-01, -3.2952e-01, -1.0596e-01,  1.3604e+00,  1.8881e-01],
         [-1.7524e-01,  8.3306e-01,  1.0397e+00, -6.9338e-01, -1.2310e+00],
         [-1.6071e+00, -4.6016e-01,  6.5157e-01, -2.9964e-01,  8.4878e-02],
         [ 1.3175e-01,  1.3799e+00, -2.0456e+00,  4.4319e-01,  1.1152e+00],
         [ 1.0430e+00,  1.2995e+00,  1.6779e+00, -4.2784e-01,  1.7992e+00],
         [ 1.9181e+00,  4.7578e-01, -2.6459e-01,  4.6090e-01,  2.3222e+00],
         [ 8.9001e-01, -1.1867e+00,  3.4631e-01, -6.7855e-01,  3.3243e-01],
         [-3

In [67]:
import math

import torch
import torch.nn.functional as F

import torch
import torch.nn as nn

class PositionalEncoding(nn.Module): # for BERT and ALBERT
    def __init__(self, n_embed, max_seq_len):
        super(PositionalEncoding, self).__init__()
        
        # Create a matrix of shape (max_len, d_model) with positional encodings
        pe = torch.zeros(max_seq_len, n_embed)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        
        # Div term represents the frequency of the sine and cosine functions
        div_term = torch.exp(torch.arange(0, n_embed, 2).float() * (-torch.log(torch.tensor(10000.0)) / n_embed))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Add a batch dimension for broadcasting
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        # Register pe as a buffer, which means it's not a parameter but should be part of the state
        self.register_buffer('pe', pe)
        

        self.layer_norm = torch.nn.LayerNorm(n_embed, eps=1e-12, elementwise_affine=True) # eps: added as sqrt(var + eps) to prevent zero denominator
        self.dropout = torch.nn.Dropout(p=0.1, inplace=False) # inplace=False: do not replace the input by dropouted input
    
    def forward(self, x):
        # Add positional encoding to the input embeddings
        x = x + self.pe[:x.size(0), :]
        embeddings = self.layer_norm(x)
        embeddings = self.dropout(embeddings)
        return embeddings

class AbsolutePositionEmbedding(torch.nn.Module): # for DEBERTA
    def __init__(self, n_embed, max_seq_len):
        super(AbsolutePositionEmbedding, self).__init__()
        
        self.max_seq_len = max_seq_len
        
        self.position_embeddings = nn.Embedding(max_seq_len, n_embed)
        
    def forward(self, input_ids):
        # Get the position IDs from the input IDs
        position_ids = torch.arange(0, self.max_seq_len, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        
        # Get the position embeddings
        position_embeddings = self.position_embeddings(position_ids)
        
        return position_embeddings
    
class BertEmbeddings(torch.nn.Module):
    def __init__(self, vocab_size, n_embed): # n_embed = 3, max_seq_len = 16
        super().__init__()

        self.word_embeddings = torch.nn.Embedding(vocab_size, n_embed) # number of words is length of text, each words has length n_embed

    def forward(self, x):
        words_embeddings = self.word_embeddings(x)

        return words_embeddings
    
class ALBertEmbeddings(torch.nn.Module):
    def __init__(self, vocab_size, n_embed, n_hid = 3): # n_embed = 3
        super().__init__()

        self.hid_embeddings = torch.nn.Embedding(vocab_size, n_hid) # number of words is length of text, each words has length n_embed

        self.word_embeddings = torch.nn.Embedding(n_hid, n_embed) # number of words is length of text, each words has length n_embed
    def forward(self, x):
        hid_embeddings = self.hid_embeddings(x)
        
        words_embeddings = self.word_embeddings(hid_embeddings)

        return words_embeddings
    
class DEBertAEmbeddings(torch.nn.Module):
    def __init__(self, vocab_size, max_seq_len, n_embed_word = 3, n_embed_p = 1): 
        super().__init__()

        self.word_embeddings = torch.nn.Embedding(vocab_size, n_embed_word) # number of words is length of text, each words has length n_embed

        self.abposit_embeddings = AbsolutePositionEmbedding(n_embed_p, max_seq_len)
    def forward(self, x):
        
        words_embeddings = self.word_embeddings(x)
        
        abposits_embeddings = self.abposit_embeddings(x)

        return torch.cat((words_embeddings, abposits_embeddings), dim = 2)


class BertAttentionHead(torch.nn.Module):
    """
    A single attention head in MultiHeaded Self Attention layer.
    The idea is identical to the original paper ("Attention is all you need"),
    however instead of implementing multiple heads to be evaluated in parallel we matrix multiplication,
    separated in a distinct class for easier and clearer interpretability
    """

    def __init__(self, head_size, dropout, n_embed): # dropout = 0.1, n_embed = 3
        super().__init__()

        self.query = torch.nn.Linear(in_features=n_embed, out_features=head_size)
        self.key = torch.nn.Linear(in_features=n_embed, out_features=head_size)
        self.values = torch.nn.Linear(in_features=n_embed, out_features=head_size)

        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, mask):
        # B, Seq_len, N_embed
        B, seq_len, n_embed = x.shape

        q = self.query(x)
        k = self.key(x)
        v = self.values(x)

        weights = (q @ k.transpose(-2, -1)) / math.sqrt(n_embed)  # (B, Seq_len, Seq_len)
        weights = weights.masked_fill(mask == 0, -1e9)  # mask out not attended tokens

        scores = F.softmax(weights, dim=-1)
        scores = self.dropout(scores)

        context = scores @ v

        return context


class BertSelfAttention(torch.nn.Module):
    """
    MultiHeaded Self-Attention mechanism as described in "Attention is all you need"
    """

    def __init__(self, n_heads, dropout, n_embed): # , n_heads = 1, dropout = 0.1, n_embed = 3
        super().__init__()

        head_size = n_embed // n_heads

        n_heads = n_heads

        self.heads = torch.nn.ModuleList([BertAttentionHead(head_size, dropout, n_embed) for _ in range(n_heads)])

        self.proj = torch.nn.Linear(head_size * n_heads, n_embed)  # project from multiple heads to the single space

        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, mask):
        context = torch.cat([head(x, mask) for head in self.heads], dim=-1)

        proj = self.proj(context)

        out = self.dropout(proj)

        return out


class FeedForward(torch.nn.Module):
    def __init__(self, dropout, n_embed): # dropout=0.1, n_embed=3
        super().__init__()

        self.ffwd = torch.nn.Sequential(
            torch.nn.Linear(n_embed, 4 * n_embed),
            torch.nn.GELU(),
            torch.nn.Linear(4 * n_embed, n_embed),
            torch.nn.Dropout(dropout),
        )

    def forward(self, x):
        out = self.ffwd(x)

        return out


class BertLayer(torch.nn.Module):
    """
    Single layer of BERT transformer model
    """

    def __init__(self, n_heads, dropout, n_embed): # n_heads=1, dropout=0.1, n_embed=3
        super().__init__()

        # unlike in the original paper, today in transformers it is more common to apply layer norm before other layers
        # this idea is borrowed from Andrej Karpathy's series on transformers implementation
        self.layer_norm1 = torch.nn.LayerNorm(n_embed)
        self.self_attention = BertSelfAttention(n_heads, dropout, n_embed)

        self.layer_norm2 = torch.nn.LayerNorm(n_embed)
        self.feed_forward = FeedForward(dropout, n_embed)

    def forward(self, x, mask):
        x = self.layer_norm1(x)
        x = x + self.self_attention(x, mask)

        x = self.layer_norm2(x)
        out = x + self.feed_forward(x)

        return out


class BertEncoder(torch.nn.Module):
    def __init__(self, n_layers, n_heads, dropout, n_embed): # n_layers=2, n_heads=1, dropout=0.1, n_embed=3
        super().__init__()

        self.layers = torch.nn.ModuleList([BertLayer(n_heads, dropout, n_embed) for _ in range(n_layers)])

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)

        return x


class BertPooler(torch.nn.Module):
    def __init__(self, dropout, n_embed): # dropout=0.1, n_embed=3
        super().__init__()

        self.dense = torch.nn.Linear(in_features=n_embed, out_features=n_embed)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        pooled = self.dense(x)
        out = self.activation(pooled)

        return out


class NanoBERT(torch.nn.Module):
    """
    NanoBERT is a almost an exact copy of a transformer decoder part described in the paper "Attention is all you need"
    This is a base model that can be used for various purposes such as Masked Language Modelling, Classification,
    Or any other kind of NLP tasks.
    This implementation does not cover the Seq2Seq problem, but can be easily extended to that.
    """

    def __init__(self, vocab_size, n_layers, n_heads, dropout, n_embed, max_seq_len): # n_layers=2, n_heads=1, dropout=0.1, n_embed=4, max_seq_len = 16
        """

        :param vocab_size: size of the vocabulary that tokenizer is using
        :param n_layers: number of BERT layer in the model (default=2)
        :param n_heads: number of heads in the MultiHeaded Self Attention Mechanism (default=1)
        :param dropout: hidden dropout of the BERT model (default=0.1)
        :param n_embed: hidden embeddings dimensionality (default=3)
        :param max_seq_len: max length of the input sequence (default=16)
        """
        super().__init__()

        self.embedding = ALBertEmbeddings(vocab_size, n_embed)
#         self.embedding = BertEmbeddings(vocab_size, n_embed)
        
        self.position = PositionalEncoding(n_embed, max_seq_len)

        self.encoder = BertEncoder(n_layers, n_heads, dropout, n_embed)

        self.pooler = BertPooler(dropout, n_embed)

    def forward(self, x):
        # attention masking for padded token
        # (batch_size, seq_len, seq_len)
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1)

        embeddings = self.embedding(x)
        
        position_embeddings = self.position(embeddings)

        encoded = self.encoder(position_embeddings, mask)

        pooled = self.pooler(encoded)
        return pooled


class BertMix(torch.nn.Module):
    """
    This is a wrapper on the base NanoBERT that is used for classification task
    One can use this as an example of how to extend and apply nano-BERT to similar custom tasks
    This layer simply adds one additional dense layer for classification
    """

    def __init__(self, vocab_size, n_layers=2, n_heads=1, dropout=0.1, n_embed=4, max_seq_len=16, n_classes=2): # n_layers=2, n_heads=1, dropout=0.1, n_embed=3, max_seq_len=16, n_classes=2
        super().__init__()
        self.nano_bert = NanoBERT(vocab_size, n_layers, n_heads, dropout, n_embed, max_seq_len)

        self.classifier = torch.nn.Linear(in_features=n_embed, out_features=n_classes)
        self.mlm = torch.nn.Linear(in_features=n_embed, out_features=vocab_size)

    def forward(self, input_ids):
        embeddings = self.nano_bert(input_ids)

        r_cls = self.classifier(embeddings)
        r_mlm = self.mlm(embeddings)
        return r_cls, r_mlm
    
class BertMix3(torch.nn.Module):
    """
    This is a wrapper on the base NanoBERT that is used for classification task
    One can use this as an example of how to extend and apply nano-BERT to similar custom tasks
    This layer simply adds one additional dense layer for classification
    """

    def __init__(self, vocab_size, n_layers=2, n_heads=1, dropout=0.1, n_embed=4, max_seq_len=128, n_classes=2): # n_layers=2, n_heads=1, dropout=0.1, n_embed=3, max_seq_len=16, n_classes=2
        super().__init__()
        self.nano_bert = NanoBERT(vocab_size, n_layers, n_heads, dropout, n_embed, max_seq_len)

        self.classifier = torch.nn.Linear(in_features=n_embed, out_features=n_classes)
        self.mlm = torch.nn.Linear(in_features=n_embed, out_features=vocab_size)
        self.nsp = torch.nn.Linear(in_features=n_embed, out_features=n_classes)

    def forward(self, input_ids):
        embeddings = self.nano_bert(input_ids)

        r_cls = self.classifier(embeddings)
        r_mlm = self.mlm(embeddings)
        r_nsp = self.nsp(embeddings)
        return r_cls, r_mlm, r_nsp

In [19]:
bert = BertMix3(
    vocab_size=len(tokenizer.vocab),
    n_layers=3,
    n_heads=1,
    max_seq_len=MAX_SEQ_LEN,
    n_classes=NUM_CLASS,
    n_embed = 4
).to(device)
bert

BertMix3(
  (nano_bert): NanoBERT(
    (embedding): ALBertEmbeddings(
      (hid_embeddings): Embedding(3665, 3)
      (word_embeddings): Embedding(3, 4)
    )
    (position): PositionalEncoding(
      (layer_norm): LayerNorm((4,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layers): ModuleList(
        (0-2): 3 x BertLayer(
          (layer_norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
          (self_attention): BertSelfAttention(
            (heads): ModuleList(
              (0): BertAttentionHead(
                (query): Linear(in_features=4, out_features=4, bias=True)
                (key): Linear(in_features=4, out_features=4, bias=True)
                (values): Linear(in_features=4, out_features=4, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
            (proj): Linear(in_features=4, out_features=4, bias=True)
            (dropout)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(bert)

In [ ]:
dataloader.steps('train')
dataloader.get_split('train')

In [ ]:
import random
def mask_text(cmt, vacab_size): # cmt : a comment
    mps = []
    mcmt = cmt.clone()
    unique_elements, counts = torch.unique(cudata[i], return_counts=True)
    n_sp = len(tokenizer.special_tokens) # number of special_tokens = 7
    m_range = (MAX_SEQ_LEN - counts[0].item()) * 0.15 # range of masking indice
#     print(m_range, counts[0].item())
    while len(mps) < m_range:
        temp = random.randint(0, MAX_SEQ_LEN - 1)
        if mcmt[temp] > n_sp - 1:
            mps.append(temp)
            mcmt[temp] = 0 # set to mask
    temp_m = random.sample(range(0, len(mps) - 1), 2 * int(m_range * 0.1 + 1)) # fetch some masked words to their original value
    half = int(len(temp_m)/2)
    tmps = torch.tensor(mps) # tensorlize mps to get location of masks to be changed
    mcmt[tmps[temp_m[:half]]] = cmt[temp_m[:half]] # 10% percent original
    mcmt[tmps[temp_m[half:]]] = torch.tensor([random.randint(n_sp, vacab_size - 1) for i in range(half)]).to(device) # 10% percent random vocab
    return mps, mcmt


In [61]:
def spit_cmt(cmt): # split  comment into sentences
    bg = torch.where(cmt==5)[0] # begin at '[SOS]'
    ed = torch.where(cmt==6)[0] # end at '.'
    spit = []
    n_st = len(bg) # number of next sentence prediction task in this comment
    for i in range(n_st):
        sts = cmt[bg[i].item():ed[i].item()+1]
        spit.append(sts)
    return spit, n_st
def pad_nsp(nsp, max_seq_len): # add '[CLS]' and padding on given composed 2 sentences in GPU
#     print('len nsp = ', len(nsp))
    return torch.cat((torch.tensor([0]).to(device), nsp, torch.ones(max_seq_len - len(nsp) - 1).to(device)), 0)
def nsp_gen(spit, n_st, max_seq_len): # generate nsp input      
    nsp = torch.zeros((2 * (n_st - 1), max_seq_len)).to(device)
    if n_st < 3:
        nsp = torch.zeros((1, max_seq_len)).to(device)
#     print(nsp[:10], n_st)
    for id_i in range(n_st - 1):
        nsp[2 * id_i] = pad_nsp(torch.cat((spit[id_i], spit[id_i + 1]), 0), max_seq_len) # a sentence followed by the next
        if n_st > 2:
            id_s = random.randint(0, n_st - 1) # selected id for nsp
            while id_s == id_i + 1 or id_s == id_i: # excluded the next sentence
                id_s = random.randint(0, n_st - 1)
            nsp[2 * id_i + 1] = pad_nsp(torch.cat((spit[id_i], spit[id_s]), 0), max_seq_len) # a sentence not followed by the next
    return nsp.long().to(device)

In [ ]:
history_pretrain = {
    'train_losses': [],
    'val_losses': [],
    'train_acc': [],
    'val_acc': [],
    'train_f1': [],
    'val_f1': []
}
history = {
    'train_losses': [],
    'val_losses': [],
    'train_acc': [],
    'val_acc': [],
    'train_f1': [],
    'val_f1': []
}

### pretrain task, 25 min/epoch

In [ ]:
optimizer = torch.optim.Adam(bert.parameters(), lr=LEARNING_RATE)
NUM_EPOCHS = 15 # epochs for pretrain task
vacab_size = len(tokenizer.vocab)
for i in range(NUM_EPOCHS):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0

    bert.train()
    for step, batch in enumerate(tqdm(dataloader.get_split('train'), total=dataloader.steps('train'), position=0)):
        loss = 0
        cudata = batch['input_ids'].to(device) # put the dat in the whole batch to gpu
        # MLM part
        for i in range(len(cudata)): # BATCH_SIZE = len(cudata)
            mps, mcmt = mask_text(cudata[i], vacab_size)
            lmps = len(mps)
            _, r_mlm, _ = bert(mcmt.unsqueeze(dim=0)) # (Batch, Seq_Len, len(vocab))
            MCMT = r_mlm[0][mps, :] # fectch the predicted value from masked input
            predm = cudata[i][mps].long() # fectch the real word correspond to the masked input
            for i in range(lmps):
                loss += F.cross_entropy(MCMT.to(device), predm.to(device)) / lmps
        
        # NSP part
        for i in range(len(cudata)): 
            spit, n_st = spit_cmt(cudata[i]) # (Batch, Seq_Len, len(vocab)), number of sentences
            if n_st > 1: # calculate NSP loss if a comment has more than 1 sentence
                predn = torch.tensor([1, 0] * (n_st - 1)).to(device) # label of next/not next composed sentences
                if n_st < 3:
                    predn = torch.tensor([1]).to(device)
                _, _, r_nsp = bert(nsp_gen(spit, n_st, MAX_SEQ_LEN))
    #             print(n_st)
    #             print(r_nsp[:, 0, :].shape, predn.shape)
    #             print(r_nsp[:, 0, :], predn)
                loss +=  F.cross_entropy(r_nsp[:, 0, :], predn)
        print('train_loss:', '%.2f'%loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
    history_pretrain['train_losses'].append(train_loss / dataloader.steps("train"))
    val_loss = 0.0

In [ ]:
PATH = 'imdb_pre_para.pth'# parameter for pretrain task
torch.save(bert.state_dict(), PATH)

### Fine-tuning on text classification downstream task, 20s/epoch

In [ ]:
BATCH_SIZE_F = 16

In [ ]:
dataloader_F = IMDBDataloader(data, test_data, tokenizer, encode_label, batch_size=BATCH_SIZE_F)

In [ ]:
NUM_EPOCHS_F = 200

for i in range(NUM_EPOCHS_F):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    bert.train()
    for step, batch in enumerate(tqdm(dataloader_F.get_split('train'), total=dataloader_F.steps('train'))):
        r_cls, _, _ = bert(batch['input_ids'].to(device)) # (B, Seq_Len, 2)

        probs = F.softmax(r_cls[:, 0, :], dim=-1).cpu()# fetch the result from the first word in the text as output
        pred = torch.argmax(probs, dim=-1) # (B)
        train_preds += pred.detach().tolist()
        train_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(r_cls[:, 0, :].cpu(), batch['label_ids'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    val_loss = 0.0
    val_preds = []
    val_labels = []

    bert.eval()
    for step, batch in enumerate(tqdm(dataloader.get_split('val'), total=dataloader.steps('val'))):
        r_cls, _, _ = bert(batch['input_ids'].to(device))

        probs = F.softmax(r_cls[:, 0, :], dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        val_preds += pred.detach().tolist()
        val_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(r_cls[:, 0, :].cpu(), batch['label_ids'])

        val_loss += loss.item()

    history['train_losses'].append(train_loss)
    history['val_losses'].append(val_loss)
    history['train_acc'].append(accuracy_score(train_labels, train_preds))
    history['val_acc'].append(accuracy_score(val_labels, val_preds))
    history['train_f1'].append(f1_score(train_labels, train_preds))
    history['val_f1'].append(f1_score(val_labels, val_preds))

    print()
    print(f'Train loss: {train_loss / dataloader.steps("train")} | Val loss: {val_loss / dataloader.steps("val")}')
    print(f'Train acc: {accuracy_score(train_labels, train_preds)} | Val acc: {accuracy_score(val_labels, val_preds)}')
    print(f'Train f1: {f1_score(train_labels, train_preds)} | Val f1: {f1_score(val_labels, val_preds)}')

In [ ]:
PATH = 'imdb_tcl_para_BERT.pth'# parameter for text classification
torch.save(bert.state_dict(), PATH)

In [ ]:
def plot_results(history, do_val=True):
    fig, ax = plt.subplots(figsize=(8, 8))

    x = list(range(0, len(history['train_losses'])))

    # loss

    ax.plot(x, history['train_losses'], label='train_loss')

    if do_val:
        ax.plot(x, history['val_losses'], label='val_loss')

    plt.title('Train / Validation Loss')
    plt.legend(loc='upper right')

    # accuracy

    fig, ax = plt.subplots(figsize=(8, 8))

    ax.plot(x, history['train_acc'], label='train_acc')

    if do_val:
        ax.plot(x, history['val_acc'], label='val_acc')

    plt.title('Train / Validation Accuracy')
    plt.legend(loc='upper right')

    # f1-score

    fig, ax = plt.subplots(figsize=(8, 8))

    ax.plot(x, history['train_f1'], label='train_f1')

    if do_val:
        ax.plot(x, history['val_f1'], label='val_f1')

    plt.title('Train / Validation F1')
    plt.legend(loc='upper right')

    fig.show()

In [ ]:
plot_results(history)

In [ ]:
logits[0]

In [ ]:
test_loss = 0.0
test_preds = []
test_labels = []

bert.eval()
for step, batch in enumerate(tqdm(dataloader.get_split('test'), total=dataloader.steps('test'))):
    logits = bert(batch['input_ids'].to(device))[0]

    probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
    pred = torch.argmax(probs, dim=-1) # (B)
    test_preds += pred.detach().tolist()
    test_labels += [l.item() for l in batch['label_ids']]

    loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

    test_loss += loss.item()

print()
print(f'Test loss: {test_loss / dataloader.steps("test")}')
print(f'Test acc: {accuracy_score(test_labels, test_preds)}')
print(f'Test f1: {f1_score(test_labels, test_preds)}')

# Interpreting and visualizing the results

In [ ]:
def get_attention_scores(model, input_ids):
    """
    This is just a wrapper to easily access attention heads of the last layer
    """

    mask = (input_ids > 0).unsqueeze(1).repeat(1, input_ids.size(1), 1)

    embed = model.nano_bert.embedding(input_ids)

    # can be any layer, and we can also control what to do with output for each layer (aggregate, sum etc.)
    layer = model.nano_bert.encoder.layers[-1]

    x = layer.layer_norm1(embed)

    B, seq_len, n_embed = x.shape

    # if have more than 1 head, or interested in more than 1 head output just add aggregation here
    head = layer.self_attention.heads[0]

    # this is just a part of the single head that does all the computations (same code is present in AttentionHead)
    q = head.query(x)
    k = head.key(x)
    v = head.values(x)

    weights = (q @ k.transpose(-2, -1)) / math.sqrt(n_embed)  # (B, Seq_len, Seq_len)
    weights = weights.masked_fill(mask == 0, -1e9)  # mask out not attended tokens

    scores = F.softmax(weights, dim=-1)

    return scores

In [ ]:
test_dataloader = IMDBDataloader(data, test_data, tokenizer, encode_label, batch_size=1)

In [ ]:
def plot_parallel(matrix, tokens):
    # Set figsize
    plt.figure(figsize=(12, 8))

    input_len = len(tokens)

    # Vertical lines
    plt.axvline(x=1, color='black', linestyle='--', linewidth=1)
    plt.axvline(x=5, color='black', linestyle='--', linewidth=1)

    # Add the A and B
    plt.text(1, input_len + 1, 'A', fontsize=12, color='black', fontweight='bold')
    plt.text(5, input_len + 1, 'B', fontsize=12, color='black', fontweight='bold')

    for i in range(input_len):
        for j in range(input_len):
            # Add the line to the plot
            plt.plot([1, 5], [i, j], marker='o', label='token', color='blue', linewidth=5 * matrix[i][j])

            plt.text(
                1 - 0.18,  # x-axis position
                i,  # y-axis position
                tokens[i],  # Text
                fontsize=8,  # Text size
                color='black',  # Text color,
            )

            plt.text(
                5 + 0.06,  # x-axis position
                j,  # y-axis position
                tokens[j],  # Text
                fontsize=8,  # Text size
                color='black',  # Text color
            )
        break

    plt.title(f'Attention scores \n\n\n')

    plt.yticks([])  # Remove y-axis
    plt.box(False)  # Remove the bounding box around plot
    plt.show()  # Display the chart

In [ ]:
# examples with less than 16 words are easier to visualize, so focus on them
examples_ids = []
for i, v in enumerate(test_dataloader.splits['test']):
    if len(v) <= 16:
        examples_ids.append(i)
print(examples_ids)

In [ ]:
tokens

In [ ]:
for sample_index in examples_ids:
    # extract example, decode to tokens and get the sequence length (ingoring padding)
    test_tokenized_batch = test_dataloader.peek_index_tokenized(index=sample_index, split='test')
    tokens = tokenizer.decode([t.item() for t in test_tokenized_batch['input_ids'][0] if (t != 0 and t.item() != 1)], ignore_special=False).split(' ')[:MAX_SEQ_LEN]
    seq_len = len(tokens)

    # calculate attention scores
    att_matrix = get_attention_scores(bert, test_tokenized_batch['input_ids'].to(device))[0, :seq_len, :seq_len]

    plot_parallel(att_matrix, tokens=tokens)